In [1]:
import pandas as pd
import numpy as np
import math
import re
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
cosine = pd.read_csv('pub_cosine_similarity_category.csv')
data = pd.read_csv('Final_Data_For_NLP.csv')
data = data.rename(columns={'Unnamed: 0': 'Doc_ID'})
cosine = cosine.rename(columns={'Unnamed: 0': 'Doc_ID'})
combined = pd.merge(data, cosine, on = ['Doc_ID'])

In [4]:
data

,Doc_ID,Name,descriptions,titles,date,source,citations,Location
0,0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,NaN,NaN,NaN,NaN,CEC Website
1,1,aditya johri,aditya johri studies use information communica...,NaN,NaN,NaN,NaN,CEC Website
2,2,ahmed bin zaman,ever tell story let say friends called zoo bad...,NaN,NaN,NaN,NaN,CEC Website
3,3,alexander brodsky,alex brodsky joined george mason university 19...,NaN,NaN,NaN,NaN,CEC Website
4,4,alexander levis,alexander h levis university professor electri...,NaN,NaN,NaN,NaN,CEC Website
...,...,...,...,...,...,...,...,...
16104,16104,yue cheng,distributed systems storage systems cloud comp...,NaN,NaN,NaN,NaN,LinkedIn About
16105,16105,yue wang,currently research assistant professor electri...,NaN,NaN,NaN,NaN,LinkedIn About
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About
16107,16107,ziwei zhu,assistant professor department computer scienc...,NaN,NaN,NaN,NaN,LinkedIn About


In [5]:
CEC_b = data[data['Location'] == 'CEC Website']
GS = data[data['Location'] == 'Google Scholar']
ASC = data[data['Location'] == 'Academic Search Complete']
LP = data[data['Location'] == 'LinkedIn Publication']
LA = data[data['Location'] == 'LinkedIn About']


In [6]:
CEC_b = CEC_b[CEC_b['descriptions'].notna()]
CEC_b['loc'] = CEC_b['Location']+' '+CEC_b['Name']
CEC_b.to_csv('loadable_CEC_Bio.csv')
CEC_b

,Doc_ID,Name,descriptions,titles,date,source,citations,Location,loc
0,0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,NaN,NaN,NaN,NaN,CEC Website,CEC Website abolfazl safikhani
1,1,aditya johri,aditya johri studies use information communica...,NaN,NaN,NaN,NaN,CEC Website,CEC Website aditya johri
2,2,ahmed bin zaman,ever tell story let say friends called zoo bad...,NaN,NaN,NaN,NaN,CEC Website,CEC Website ahmed bin zaman
3,3,alexander brodsky,alex brodsky joined george mason university 19...,NaN,NaN,NaN,NaN,CEC Website,CEC Website alexander brodsky
4,4,alexander levis,alexander h levis university professor electri...,NaN,NaN,NaN,NaN,CEC Website,CEC Website alexander levis
...,...,...,...,...,...,...,...,...,...
236,236,yutao zhong,yutao zhong contributes research academic exce...,NaN,NaN,NaN,NaN,CEC Website,CEC Website yutao zhong
237,237,zelalem eshete,zelalem eshete taught various colleges univers...,NaN,NaN,NaN,NaN,CEC Website,CEC Website zelalem eshete
238,238,zhisheng yan,zhisheng yan assistant professor department in...,NaN,NaN,NaN,NaN,CEC Website,CEC Website zhisheng yan
239,239,ziwei zhu,ziwei zhu broadly interested data mining machi...,NaN,NaN,NaN,NaN,CEC Website,CEC Website ziwei zhu


In [7]:
GS = GS[GS['titles'].notna()]
GS = GS.replace({np.nan: 'N/A'})
GS.to_csv('loadable_GS_Doc.csv')

In [8]:
ASC = ASC.replace({np.nan: 'N/A'})
ASC.to_csv('loadable_ASC_Doc.csv')

In [9]:
LP = LP[LP['titles'].notna()]
LP = LP.replace({np.nan: 'N/A'})
LP.to_csv('loadable_L_Doc.csv')

In [10]:
LA = LA[LA['descriptions'].notna()]
LA['loc'] = LA['Location']+' '+LA['Name']
LA.to_csv('loadable_L_about.csv')

In [11]:
cosine

,Doc_ID,Name,descriptions,similar_categories,category_score
0,0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,"['statistical modeling', 'stochastic modeling'...","[0.16037507477489607, 0.14433756729740646, 0.1..."
1,1,aditya johri,aditya johri study use information communicati...,"['bioengineering', 'engineering computing stud...","[0.15474611514754325, 0.13257401008712377, 0.1..."
2,2,ahmed bin zaman,ever tell story let say friend call zoo bad re...,"['neural interfaces', 'architecture', 'hardwar...","[0.158113883008419, 0.1224744871391589, 0.1217..."
3,3,alexander brodsky,alex brodsky join george mason university 1993...,"['subsystems', 'neural networks', 'neuroscienc...","[0.18519709381740776, 0.136147096098501, 0.113..."
4,4,alexander levis,alexander h levis university professor electri...,"['subsystems', 'bioengineering', 'neuroscience...","[0.1858288839247533, 0.153301007957441, 0.1517..."
...,...,...,...,...,...
16104,16104,yue cheng,distribute system storage system cloud compute...,"['subsystems', 'neural networks', 'neuroscienc...","[0.44038550605054433, 0.30062323854087214, 0.2..."
16105,16105,yue wang,currently research assistant professor electri...,"['wireless sensing', 'wireless', 'wireless net...","[0.20257904182092135, 0.19134594929397597, 0.1..."
16106,16106,zhisheng yan,research interest broadly lie broad area mobil...,"['subsystems', 'neural networks', 'neuroscienc...","[0.3872983346207417, 0.24404676504598816, 0.23..."
16107,16107,ziwei zhu,assistant professor department computer scienc...,"['subsystems', 'intelligence c4i cyber', 'art...","[0.19364916731037085, 0.1825741858350554, 0.18..."


In [12]:
combined

,Doc_ID,Name_x,descriptions_x,titles,date,source,citations,Location,Name_y,descriptions_y,similar_categories,category_score
0,0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,NaN,NaN,NaN,NaN,CEC Website,abolfazl safikhani,abolfazl safikhani currently assistant profess...,"['statistical modeling', 'stochastic modeling'...","[0.16037507477489607, 0.14433756729740646, 0.1..."
1,1,aditya johri,aditya johri studies use information communica...,NaN,NaN,NaN,NaN,CEC Website,aditya johri,aditya johri study use information communicati...,"['bioengineering', 'engineering computing stud...","[0.15474611514754325, 0.13257401008712377, 0.1..."
2,2,ahmed bin zaman,ever tell story let say friends called zoo bad...,NaN,NaN,NaN,NaN,CEC Website,ahmed bin zaman,ever tell story let say friend call zoo bad re...,"['neural interfaces', 'architecture', 'hardwar...","[0.158113883008419, 0.1224744871391589, 0.1217..."
3,3,alexander brodsky,alex brodsky joined george mason university 19...,NaN,NaN,NaN,NaN,CEC Website,alexander brodsky,alex brodsky join george mason university 1993...,"['subsystems', 'neural networks', 'neuroscienc...","[0.18519709381740776, 0.136147096098501, 0.113..."
4,4,alexander levis,alexander h levis university professor electri...,NaN,NaN,NaN,NaN,CEC Website,alexander levis,alexander h levis university professor electri...,"['subsystems', 'bioengineering', 'neuroscience...","[0.1858288839247533, 0.153301007957441, 0.1517..."
...,...,...,...,...,...,...,...,...,...,...,...,...
16104,16104,yue cheng,distributed systems storage systems cloud comp...,NaN,NaN,NaN,NaN,LinkedIn About,yue cheng,distribute system storage system cloud compute...,"['subsystems', 'neural networks', 'neuroscienc...","[0.44038550605054433, 0.30062323854087214, 0.2..."
16105,16105,yue wang,currently research assistant professor electri...,NaN,NaN,NaN,NaN,LinkedIn About,yue wang,currently research assistant professor electri...,"['wireless sensing', 'wireless', 'wireless net...","[0.20257904182092135, 0.19134594929397597, 0.1..."
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,"['subsystems', 'neural networks', 'neuroscienc...","[0.3872983346207417, 0.24404676504598816, 0.23..."
16107,16107,ziwei zhu,assistant professor department computer scienc...,NaN,NaN,NaN,NaN,LinkedIn About,ziwei zhu,assistant professor department computer scienc...,"['subsystems', 'intelligence c4i cyber', 'art...","[0.19364916731037085, 0.1825741858350554, 0.18..."


In [43]:
no_def = pd.read_csv('blank_defintions.csv')
no_def

,empty
0,cybersecurity
1,biomaterials nanomedicine
2,neuroengineering
3,5g
4,virtualization
5,geosynthetics
6,electrocatalysts
7,plasonics nanophotonics
8,stylometry
9,neuroinformatics


In [44]:
cosine = pd.read_csv('pub_cosine_similarity_area.csv')
data = pd.read_csv('Final_Data_For_NLP.csv')
data = data.rename(columns={'Unnamed: 0': 'Doc_ID'})
cosine = cosine.rename(columns={'Unnamed: 0': 'Doc_ID'})
combined = pd.merge(data, cosine, on = ['Doc_ID'])

In [45]:
#Run for publication areas
combined['similar_area'] = combined['similar_area'].replace(["\[","'","]"],'', regex=True)
combined['similar_area'] = combined['similar_area'].str.split(", ")
combined['area_score'] = combined['area_score'].replace(["\[","'","]"],'', regex=True)
combined['area_score'] = combined['area_score'].str.split(", ")
combined = combined.explode(['similar_area','area_score'])
combined['area_score'] = pd.to_numeric(combined['area_score'])
combined = combined.loc[~combined['similar_area'].isin(no_def['empty'])]
temp_pub_area = combined
combined.to_csv('loadable_cosine_pub_data_area_full.csv')
combined = combined[combined['area_score'] >= 0.2]
combined.to_csv('loadable_cosine_pub_data_area.csv')
combined

,Doc_ID,Name_x,descriptions_x,titles,date,source,citations,Location,Name_y,descriptions_y,similar_area,area_score
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,computer networks communications,0.244707
16,16,özlem uzuner,özlem uzuner specializes natural language proc...,NaN,NaN,NaN,NaN,CEC Website,Özlem Uzuner,Özlem Uzuner specialize natural language proce...,communitybased healthcare,0.221428
17,17,ben seiyon lee,ben lee research interests include 1 computati...,NaN,NaN,NaN,NaN,CEC Website,ben seiyon lee,ben lee research interest include 1 computatio...,statistical modeling,0.218218
17,17,ben seiyon lee,ben lee research interests include 1 computati...,NaN,NaN,NaN,NaN,CEC Website,ben seiyon lee,ben lee research interest include 1 computatio...,stochastic modeling,0.218218
88,88,irina hashmi,irina hashmi work bridges space biological sci...,NaN,NaN,NaN,NaN,CEC Website,irina hashmi,irina hashmi work bridge space biological scie...,computational biomedicine,0.265580
...,...,...,...,...,...,...,...,...,...,...,...,...
16039,16039,lishan yang,tenure track assistant professor department co...,NaN,NaN,NaN,NaN,LinkedIn About,lishan yang,tenure track assistant professor department co...,software engineering,0.223885
16045,16045,massimiliano albanese,computer scientist working field secure inform...,NaN,NaN,NaN,NaN,LinkedIn About,massimiliano albanese,computer scientist working field secure inform...,artificial intelligence,0.252352
16045,16045,massimiliano albanese,computer scientist working field secure inform...,NaN,NaN,NaN,NaN,LinkedIn About,massimiliano albanese,computer scientist working field secure inform...,computer networks communications,0.231982
16057,16057,myeong lee,interested understanding dynamics local commun...,NaN,NaN,NaN,NaN,LinkedIn About,myeong lee,interested understanding dynamic local communi...,computer networks communications,0.250278


In [46]:
are = combined['similar_area'].unique()
are

array(['computer networks communications', 'communitybased healthcare',
       'statistical modeling', 'stochastic modeling',
       'computational biomedicine', 'software engineering',
       'visual computing',
       'bioinformatics biostatistics computational biology',
       'artificial intelligence', 'data analytics',
       'computational fluid dynamics', 'education',
       'biomedical imaging devices', 'design results clinical studies',
       'electrical engineering', 'geotechnical engineering',
       'structural engineering', 'multiscale tribology contact mechanics',
       'optimization techniques', 'humanities',
       'experimental understanding disease mechanisms',
       'applied probability statistics',
       'environmental water resources engineering', 'finance',
       'computer engineering', 'systems engineering',
       'nanomaterials photonics synthesis',
       'construction engineering management', 'civilian government ngos',
       'manufacturing', 'theory co

In [47]:
cosine = pd.read_csv('pub_cosine_similarity_category.csv')
data = pd.read_csv('Final_Data_For_NLP.csv')
data = data.rename(columns={'Unnamed: 0': 'Doc_ID'})
cosine = cosine.rename(columns={'Unnamed: 0': 'Doc_ID'})
combined = pd.merge(data, cosine, on = ['Doc_ID'])

In [48]:
#Run for publication categoires
combined['similar_categories'] = combined['similar_categories'].replace(["\[","'","]"],'', regex=True)
combined['similar_categories'] = combined['similar_categories'].str.split(", ")
combined['category_score'] = combined['category_score'].replace(["\[","'","]"],'', regex=True)
combined['category_score'] = combined['category_score'].str.split(", ")
combined = combined.explode(['similar_categories','category_score'])
combined = combined[combined['similar_categories'] != 'modeling nanoparticles']
combined = combined[combined['similar_categories'] != 'sustainable computing']
combined = combined[combined['similar_categories'] != 'immersive computing']
combined['similar_categories'] = combined['similar_categories'].replace('nanomaterials ','', regex=True)
combined['category_score'] = pd.to_numeric(combined['category_score'])
combined = combined.loc[~combined['similar_categories'].isin(no_def['empty'])]
temp_pub_cat = combined
combined.to_csv('loadable_cosine_pub_data_cat_full.csv')
combined = combined[combined['category_score'] >= 0.2]
combined

,Doc_ID,Name_x,descriptions_x,titles,date,source,citations,Location,Name_y,descriptions_y,similar_categories,category_score
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,subsystems,0.582772
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,neural networks,0.380820
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,neuroscience,0.356873
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,wireless networks,0.307330
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,cognitive networks,0.295507
...,...,...,...,...,...,...,...,...,...,...,...,...
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,subsystems,0.387298
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,neural networks,0.244047
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,neuroscience,0.237171
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,internet,0.230940


In [49]:
cat = combined['similar_categories'].unique()
cat
df = pd.DataFrame(are, columns = ['area'])
common = df.loc[df['area'].isin(cat)]
temp = list(common['area'])
combined = combined[combined['similar_categories'].isin(temp) == False]
combined.to_csv('loadable_cosine_pub_data_cat.csv')
combined

,Doc_ID,Name_x,descriptions_x,titles,date,source,citations,Location,Name_y,descriptions_y,similar_categories,category_score
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,subsystems,0.582772
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,neural networks,0.380820
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,neuroscience,0.356873
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,wireless networks,0.307330
6,6,ali k raz,world around us continually shaped intelligent...,NaN,NaN,NaN,NaN,CEC Website,ali k raz,world around we continually shape intelligent ...,cognitive networks,0.295507
...,...,...,...,...,...,...,...,...,...,...,...,...
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,subsystems,0.387298
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,neural networks,0.244047
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,neuroscience,0.237171
16106,16106,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,LinkedIn About,zhisheng yan,research interest broadly lie broad area mobil...,internet,0.230940


In [50]:
#Run for person area
final = pd.read_csv('cosine_similarity_area.csv')
final['similar_area'] = final['similar_area'].replace(["\[","'","]"],'', regex=True)
final['similar_area'] = final['similar_area'].str.split(", ")
final['area_score'] = final['area_score'].replace(["\[","'","]"],'', regex=True)
final['area_score'] = final['area_score'].str.split(", ")
final = final[['Name', 'similar_area', 'area_score']]
final = final.explode(['similar_area','area_score'])
final = final.dropna()
final['area_score'] = pd.to_numeric(final['area_score'])
final = final.loc[~final['similar_area'].isin(no_def['empty'])]
temp_person_area = final
final.to_csv('loadable_cosine_scores_area_full.csv')
#final = final[final['area_score'] >= 0.15]
final.to_csv('loadable_cosine_scores_area.csv')
final

,Name,similar_area,area_score
0,abolfazl safikhani,stochastic modeling,0.325851
0,abolfazl safikhani,statistical modeling,0.319243
0,abolfazl safikhani,applied probability statistics,0.129191
0,abolfazl safikhani,visual computing,0.105934
0,abolfazl safikhani,systems engineering,0.096066
...,...,...,...
240,ziyu yao,education,0.093920
240,ziyu yao,humanities,0.083557
240,ziyu yao,applied probability statistics,0.078302
240,ziyu yao,optimization techniques,0.072045


In [33]:
are = final['similar_area'].unique()
are

array(['stochastic modeling', 'statistical modeling',
       'applied probability statistics', 'visual computing',
       'systems engineering', 'computational fluid dynamics',
       'electrical engineering', 'design results clinical studies',
       'theory computation', 'structural engineering', 'education',
       'biomedical imaging devices',
       'environmental water resources engineering',
       'software engineering', 'autonomous systems',
       'experimental understanding disease mechanisms',
       'construction engineering management', 'computational biomedicine',
       'multiscale transport engineering',
       'computer networks communications',
       'multiscale tribology contact mechanics',
       'bioinformatics biostatistics computational biology',
       'artificial intelligence', 'computer engineering',
       'geotechnical engineering', 'transportation', 'data analytics',
       'human centered computing', 'national security',
       'optimization techniques',

In [51]:
#Run for person category
final = pd.read_csv('cosine_similarity_category.csv')
final['similar_categories'] = final['similar_categories'].replace(["\[","'","]"],'', regex=True)
final['similar_categories'] = final['similar_categories'].str.split(", ")
final['category_score'] = final['category_score'].replace(["\[","'","]"],'', regex=True)
final['category_score'] = final['category_score'].str.split(", ")
final = final[['Name', 'similar_categories', 'category_score']]
final = final.explode(['similar_categories','category_score'])
final = final.dropna()
final = final[final['similar_categories'] != 'modeling nanoparticles']
final = final[final['similar_categories'] != 'sustainable computing']
final = final[final['similar_categories'] != 'immersive computing']
final['similar_categories'] = final['similar_categories'].replace('nanomaterials ','', regex=True)
final['category_score'] = pd.to_numeric(final['category_score'])
final = final.loc[~final['similar_categories'].isin(no_def['empty'])]
temp_person_cat = final
#final.to_csv('loadable_cosine_scores_cat_full.csv')

final

,Name,similar_categories,category_score
0,abolfazl safikhani,stochastic modeling,0.325851
0,abolfazl safikhani,modeling,0.323252
0,abolfazl safikhani,statistical modeling,0.319243
0,abolfazl safikhani,musculoskeletal modeling,0.315320
0,abolfazl safikhani,probabilistic modeling,0.309612
...,...,...,...
240,ziyu yao,survival models,0.150570
240,ziyu yao,linguistics,0.150515
240,ziyu yao,probabilistic modeling,0.148227
240,ziyu yao,modeling,0.145376


In [52]:
cat = final['similar_categories'].unique()
cat
df = pd.DataFrame(are, columns = ['area'])
common = df.loc[df['area'].isin(cat)]
temp = list(common['area'])
final = final[final['similar_categories'].isin(temp) == False]
final.to_csv('loadable_cosine_scores_cat.csv')
final

,Name,similar_categories,category_score
0,abolfazl safikhani,modeling,0.323252
0,abolfazl safikhani,musculoskeletal modeling,0.315320
0,abolfazl safikhani,probabilistic modeling,0.309612
0,abolfazl safikhani,time series modeling,0.303266
0,abolfazl safikhani,recycled materials geo environmental modeling,0.301302
...,...,...,...
240,ziyu yao,survival models,0.150570
240,ziyu yao,linguistics,0.150515
240,ziyu yao,probabilistic modeling,0.148227
240,ziyu yao,modeling,0.145376


In [53]:
#Run for area top authors
final = pd.read_csv('top10_authors_area.csv')
final['top10'] = final['top10'].replace(["\[","'","]"],'', regex=True)
final['top10'] = final['top10'].str.split(", ")
final['top10_scores'] = final['top10_scores'].replace(["\[","'","]"],'', regex=True)
final['top10_scores'] = final['top10_scores'].str.split(", ")
final = final[['area', 'top10', 'top10_scores']]
final = final.explode(['top10','top10_scores'])
final = final.dropna()
final['top10_scores'] = pd.to_numeric(final['top10_scores'])
final = final.loc[~final['area'].isin(no_def['empty'])]
temp_person_area_au = final
final.to_csv('loadable_cosine_scores_area_au.csv')
final

,area,top10,top10_scores
0,artificial intelligence,myeong lee,0.290840
0,artificial intelligence,sadegh torabi,0.175164
0,artificial intelligence,diana wang,0.169220
0,artificial intelligence,Ã¶zlem uzuner,0.156714
0,artificial intelligence,riki morikawa,0.148942
...,...,...,...
41,transportation,elise miller hooks,0.085442
41,transportation,charles white,0.082572
41,transportation,mohan venigalla,0.081694
41,transportation,gurdip singh,0.077528


In [54]:
are = final['area'].unique()
are

array(['artificial intelligence', 'autonomous systems',
       'computer engineering', 'computer networks communications',
       'data analytics', 'electrical engineering',
       'human centered computing', 'software engineering',
       'theory computation', 'visual computing',
       'bioinformatics biostatistics computational biology',
       'biomedical imaging devices', 'communitybased healthcare',
       'computational biomedicine', 'design results clinical studies',
       'experimental understanding disease mechanisms',
       'construction engineering management',
       'environmental water resources engineering',
       'geotechnical engineering', 'structural engineering',
       'computational fluid dynamics', 'multiscale transport engineering',
       'multiscale tribology contact mechanics',
       'nanomaterials photonics synthesis',
       'applied probability statistics', 'optimization techniques',
       'simulation', 'statistical modeling', 'stochastic modeling',
 

In [55]:
#Run for category top authors
final = pd.read_csv('top10_authors_cat.csv')
final['top10'] = final['top10'].replace(["\[","'","]"],'', regex=True)
final['top10'] = final['top10'].str.split(", ")
final['top10_scores'] = final['top10_scores'].replace(["\[","'","]"],'', regex=True)
final['top10_scores'] = final['top10_scores'].str.split(", ")
final = final[['Category', 'top10', 'top10_scores']]
final = final.explode(['top10','top10_scores'])
final = final.dropna()
final['Category'] = final['Category'].replace('nanomaterials ','', regex=True)
final['top10_scores'] = pd.to_numeric(final['top10_scores'])
final = final.loc[~final['Category'].isin(no_def['empty'])]
temp_person_cat = final
#final.to_csv('loadable_cosine_scores_cat_au.csv')
final

,Category,top10,top10_scores
0,cognitive systems,janis terpenny,0.170834
0,cognitive systems,jim chen,0.149550
0,cognitive systems,amarda shehu,0.138001
0,cognitive systems,ray hong,0.131642
0,cognitive systems,thomas latoza,0.129228
...,...,...,...
563,infrastructure,alexander brodsky,0.087824
563,infrastructure,daniel barbara,0.085268
563,infrastructure,matthew amissah,0.084955
563,infrastructure,george hazelrigg,0.084763


In [56]:
cat = final['Category'].unique()
cat
df = pd.DataFrame(are, columns = ['area'])
common = df.loc[df['area'].isin(cat)]
temp = list(common['area'])
final = final[final['Category'].isin(temp) == False]
final.to_csv('loadable_cosine_scores_cat_au.csv')
final

,Category,top10,top10_scores
0,cognitive systems,janis terpenny,0.170834
0,cognitive systems,jim chen,0.149550
0,cognitive systems,amarda shehu,0.138001
0,cognitive systems,ray hong,0.131642
0,cognitive systems,thomas latoza,0.129228
...,...,...,...
563,infrastructure,alexander brodsky,0.087824
563,infrastructure,daniel barbara,0.085268
563,infrastructure,matthew amissah,0.084955
563,infrastructure,george hazelrigg,0.084763


In [16]:
#Adjust name difference from dataframe and CEC names
CEC = pd.read_csv('profiles_final_all.csv')
CEC = CEC.replace({np.nan: 'NNN/AAA'})
CEC = CEC.apply(lambda x: x.astype(str).str.lower())
CEC['Name'] = CEC['Name'].map(lambda x: re.sub(r'\W+', ' ', x))
CEC = CEC.replace({'nnn/aaa': 'N/A'})
CEC = CEC[CEC['Name'] != 'nnn aaa']
CEC.to_csv('loadable_profiles_final_all.csv')
CEC

,ID,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link
0,171.0,abolfazl safikhani,assistant professor,N/A,abolfazl safikhani is currently an assistant ...,N/A,"phd, statistics, michigan state university\n",https://cec.gmu.edu/profiles/asafikha,asafikha@gmu.edu,https://sites.google.com/site/abolfazlsafikhan...
1,98.0,aditya johri,professor,703-993-5397,aditya johri studies the use of information a...,"personalized learning,","phd, learning technologies, stanford universi...",https://cec.gmu.edu/profiles/johri,johri@gmu.edu,http://mason.gmu.edu/~johri/\n
2,27.0,ahmed bin zaman,assistant professor\ncomputer science\ncollege...,N/A,"“if they ever tell my story, let them say fri...",N/A,"phd, computer science, george mason university...",https://cec.gmu.edu/node/6886,mailto;azaman6@gmu.edu,N/A
3,31.0,alexander brodsky,professor\ncomputer science\ncollege of engine...,703-993-1529,alex brodsky joined george mason university i...,"databases,","phd, computer science, the hebrew university ...",https://cec.gmu.edu/profiles/brodsky,brodsky@gmu.edu,http://cs.gmu.edu/~brodsky/\n
4,113.0,alexander levis,"professor emeritus, electrical and computer en...",703-993-1619,alexander h. levis is university professor of ...,"command and control, multi-formalism modeling,...","scd, mechanical engineering, mit\nme, mechani...",https://cec.gmu.edu/profiles/alevis,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
236,239.0,yutao zhong,associate professor\ncomputer science\ncollege...,703-993-1773,yutao zhong contributes to research and academ...,N/A,N/A,https://cec.gmu.edu/profiles/yzhong,yzhong@gmu.edu,http://cs.gmu.edu/~yzhong/\n
237,54.0,zelalem eshete,assistant professor,N/A,zelalem eshete has taught at various colleges...,N/A,"phd, mechanical engineering, university of mar...",https://cec.gmu.edu/node/7986,zeshete@gmu.edu,N/A
238,223.0,zhisheng yan,assistant professor,N/A,zhisheng yan is assistant professor in the de...,N/A,"phd, computer science and engineering, state u...",https://cec.gmu.edu/node/6941,zyan4@gmu.edu,https://mason.gmu.edu/~zyan4/index.html\n
239,240.0,ziwei zhu,assistant professor,N/A,ziwei zhu is broadly interested in data minin...,N/A,"phd, computer science, texas a&m university\n",https://cec.gmu.edu/profiles/zzhu20,zzhu20@gmu.edu,N/A


In [17]:
person_names = pd.read_csv('loadable_profiles_final_all.csv')
person_names['Name']

0      abolfazl safikhani
1            aditya johri
2         ahmed bin zaman
3       alexander brodsky
4         alexander levis
              ...        
236           yutao zhong
237        zelalem eshete
238          zhisheng yan
239             ziwei zhu
240              ziyu yao
Name: Name, Length: 241, dtype: object

In [10]:
unique_names_cat_person = temp_person_cat['Name'].unique()
unique_names_area_person = temp_person_area['Name'].unique()
unique_names_cat_pub = temp_pub_cat['Name_x'].unique()
unique_names_area_pub = temp_pub_area['Name_x'].unique()

unique_names_cat_pub[16] == person_names['Name'][16]


True

In [11]:
df = pd.DataFrame(person_names['Name'], columns = ['Name'])
missing_names = df.loc[~df['Name'].isin(unique_names_cat_person)]
missing_names

,Name
16,özlem uzuner


In [12]:
in_cat = pd.read_csv('in_cat.csv')
in_area = pd.read_csv('in_area.csv')
not_in_cat_or_area = pd.read_csv('not_in_cat_or_area.csv')
VSE = pd.read_csv('VSE Research Portfolio Explode.csv')

In [19]:
VSE[110:]

,Unnamed: 0,Topic,Area,Category,Topic_ID,Area_ID,Category_ID
110,70,resilient & sustainable infrastructure & envir...,geotechnical engineering,geosynthetics,t2,a23,c94
111,71,resilient & sustainable infrastructure & envir...,geotechnical engineering,recycled materials and geo-environmental modeling,t2,a23,c95
112,72,resilient & sustainable infrastructure & envir...,structural engineering,structural health monitoring and inspection,t2,a24,c96
113,73,resilient & sustainable infrastructure & envir...,structural engineering,structural stress,t2,a24,c97
114,74,materials and mechanics,computational fluid dynamics,NaN,t3,a25,c98
...,...,...,...,...,...,...,...
175,130,specific business sectors,transportation,safety and security,t5,a43,c158
176,130,specific business sectors,transportation,emergency response,t5,a43,c159
177,131,specific business sectors,transportation,sustainability and resilience,t5,a43,c160
178,132,specific business sectors,transportation,system management,t5,a43,c161


In [20]:
in_cat = in_cat.apply(lambda x: x.astype(str).str.lower())
in_cat['Name'] = in_cat['Name'].map(lambda x: re.sub(r'\W+', ' ', x))
in_cat['All_Research_Interest'] = in_cat['All_Research_Interest'].map(lambda x: re.sub(r'\W+', ' ', x))
in_area = in_area.apply(lambda x: x.astype(str).str.lower())
in_area['Name'] = in_area['Name'].map(lambda x: re.sub(r'\W+', ' ', x))
in_area['All_Research_Interest'] = in_area['All_Research_Interest'].map(lambda x: re.sub(r'\W+', ' ', x))
not_in_cat_or_area = not_in_cat_or_area.apply(lambda x: x.astype(str).str.lower())
not_in_cat_or_area['Name'] = not_in_cat_or_area['Name'].map(lambda x: re.sub(r'\W+', ' ', x))
not_in_cat_or_area['All_Research_Interest'] = not_in_cat_or_area['All_Research_Interest'].map(lambda x: re.sub(r'\W+', ' ', x))
VSE['Area'] = VSE['Area'].map(lambda x: re.sub(r'\W+', ' ', x))
VSE = VSE.replace({np.nan: ''})
VSE['Category'] = VSE['Category'].map(lambda x: re.sub(r'\W+', ' ', x))

In [21]:
new_stopwords = ["A", "from", "In", "This", "We","-"] # removing Stop Words
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(new_stopwords)
#Remove stop words from titles and descriptions
stop = stopwords.words('english') #Removing Stopwords



in_cat['All_Research_Interest'] = in_cat['All_Research_Interest'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stpwrd)]))
in_area['Name'] = in_area['Name'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stpwrd)]))
not_in_cat_or_area['All_Research_Interest'] = not_in_cat_or_area['All_Research_Interest'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stpwrd)]))
VSE['Area'] = VSE['Area'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stpwrd)]))
VSE['Category'] = VSE['Category'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stpwrd)]))


In [26]:
VSE['Category'][114] = 'computational fluid dynamics'
VSE[114:]

,Topic,Area,Category,Topic_ID,Area_ID,Category_ID
114,materials and mechanics,computational fluid dynamics,computational fluid dynamics,t3,a25,c98
115,materials and mechanics,multiscale transport engineering,bio based transport,t3,a26,c99
116,materials and mechanics,multiscale transport engineering,electrical based transport,t3,a26,c100
117,materials and mechanics,multiscale transport engineering,thermal based transport,t3,a26,c101
118,materials and mechanics,multiscale tribology contact mechanics,high temperature tribology,t3,a27,c102
...,...,...,...,...,...,...
175,specific business sectors,transportation,safety security,t5,a43,c158
176,specific business sectors,transportation,emergency response,t5,a43,c159
177,specific business sectors,transportation,sustainability resilience,t5,a43,c160
178,specific business sectors,transportation,system management,t5,a43,c161


In [31]:
not_in_cat_or_area[430:]
not_in_cat_or_area = not_in_cat_or_area[not_in_cat_or_area['All_Research_Interest'] != 'nan']

In [32]:
in_cat = in_cat.loc[:, ~in_cat.columns.str.contains('^Unnamed')]
in_area = in_area.loc[:, ~in_area.columns.str.contains('^Unnamed')]
not_in_cat_or_area = not_in_cat_or_area.loc[:, ~not_in_cat_or_area.columns.str.contains('^Unnamed')]
VSE = VSE.loc[:, ~VSE.columns.str.contains('^Unnamed')]
in_cat.to_csv('loadable_in_cat.csv')
in_area.to_csv('loadable_in_area.csv')
not_in_cat_or_area.to_csv('loadable_not_in_cat_or_area.csv')
VSE.to_csv('loadable_VSE_Research_Portfolio_Explode.csv')

In [52]:
r_in_cat = list(in_cat['All_Research_Interest'].unique())
not_r_in_cat = list(not_in_cat_or_area['All_Research_Interest'].unique())
r_VSE = list(VSE['Category'].unique())
score_cat = list(temp_person_cat['similar_categories'].unique())

In [51]:
VSE

,Topic,Area,Category,Topic_ID,Area_ID,Category_ID
0,digital systems and data,artificial intelligence,cognitive systems,t0,a0,c0
1,digital systems and data,artificial intelligence,intelligent agents,t0,a0,c1
2,digital systems and data,artificial intelligence,knowledge representation reasoning,t0,a0,c2
3,digital systems and data,artificial intelligence,machine learning,t0,a0,c3
4,digital systems and data,artificial intelligence,natural language processing,t0,a0,c4
...,...,...,...,...,...,...
175,specific business sectors,transportation,safety security,t5,a43,c158
176,specific business sectors,transportation,emergency response,t5,a43,c159
177,specific business sectors,transportation,sustainability resilience,t5,a43,c160
178,specific business sectors,transportation,system management,t5,a43,c161
